<a href="https://colab.research.google.com/github/LEROYNORONHA/RetailAnalysis/blob/main/Datasets/Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
from google.colab import drive
import requests
import csv
import random
from datetime import datetime, timedelta
import pandas as pd
import string

In [4]:
# Check if drive is already mounted
if not os.path.ismount('/content/drive'):
    drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('amazon_products.csv', low_memory=False)

In [ ]:
df.drop(columns=['listPrice', 'isBestSeller', 'imgUrl', 'productURL', 'boughtInLastMonth'], inplace=True)

df.rename(columns={
    'asin': 'Product_ID',
    'title': 'Product_Desc',
    'stars': 'Product_Rating',
    'reviews': 'Product_Reviews',
    'price': 'Product_Price',
    'category_id': 'Category_ID'
}, inplace=True)

In [ ]:
df = df[df['Product_Price'] != 0]

In [ ]:
df.to_csv('products.csv', index=False)

In [ ]:
df = pd.read_csv('amazon_categories.csv', low_memory=False)

In [ ]:
df.rename(columns={
    'id': 'Category_ID',
    'category_name': 'Category_Desc'
}, inplace=True)

In [ ]:
df.to_csv('categories.csv', index=False)

In [ ]:
# Fake domain lists
fake_domains = [
    'gmail.com', 'yahoo.com', 'hotmail.com', 'aol.com', 'protonmail.com',
    'mail.com', 'icloud.com', 'outlook.com', 'live.com', 'zoho.com',
    'gmx.com', 'fastmail.com', 'tutanota.com', 'yandex.com', 'hushmail.com',
    'inbox.com', 'email.com', 'startmail.com', 'mailfence.com', 'runbox.com',
    'mail.ru', 'web.de', 'laposte.net', 'bigpond.com', 'netzero.net',
    'rediffmail.com', 'rocketmail.com', 'msn.com', 'me.com', 'usa.com',
    'optusnet.com.au', 'btinternet.com', 'shaw.ca', 'verizon.net', 'trashmail.com',
    'tempmail.com', '10minutemail.com', 'tiscali.co.uk', 'orange.fr', 'sympatico.ca',
    'juno.com', 'bellsouth.net', 'freemail.hu', 'netcourrier.com', 'telus.net',
    'uk2.net', 'cox.net', 'earthlink.net', 'safe-mail.net', 'mail2world.com'
]

companies = [
    'techcorp', 'globex', 'dynalabs', 'futurebiz', 'infinisoft',
    'skyforge', 'quantix', 'zenbyte', 'nexora', 'coretech',
    'infranix', 'codevio', 'bytecraft', 'datapulse', 'metadash',
    'infocrest', 'xentrix', 'verivue', 'tekspire', 'cyberflux',
    'novalink', 'bluepixel', 'graygate', 'bitbridge', 'hypercore',
    'synpulse', 'netspire', 'mindwave', 'aetherium', 'lumidyn',
    'zenova', 'orbitex', 'sparkline', 'avionyx', 'axonify',
    'dexatek', 'uplinx', 'corevise', 'brightleaf', 'intellisys',
    'miraplex', 'infogenix', 'cortexon', 'cybernova', 'stackbright',
    'pathwave', 'aegistron', 'voxelworks', 'intellivue', 'alphaqubit'
]

country_tlds = [
    'com', 'co.uk', 'com.au', 'ca', 'co.in', 'co.nz', 'de', 'fr', 'it', 'es',
    'nl', 'se', 'no', 'fi', 'pl', 'be', 'ch', 'at', 'pt', 'ie',
    'cz', 'sk', 'ru', 'ua', 'ro', 'bg', 'gr', 'dk', 'hu', 'lt',
    'lv', 'ee', 'tr', 'hk', 'sg', 'my', 'ph', 'th', 'vn', 'id',
    'kr', 'jp', 'cn', 'za', 'ng', 'br', 'ar', 'mx', 'cl', 'pe'
]

# Email domain customizer
def custom_domain(original_email, mode='mixed'):
    username = original_email.split('@')[0]
    if mode == 'free':
        domain = random.choice(fake_domains)
    elif mode == 'company':
        domain = f"{random.choice(companies)}.com"
    elif mode == 'country':
        domain = f"{random.choice(companies)}.{random.choice(country_tlds)}"
    else:
        domain_type = random.choice(['free', 'company', 'country'])
        return custom_domain(original_email, mode=domain_type)
    return f"{username}@{domain}"

# Fetch & create fake customers
def get_customers(num_customers=10):
    url = f'https://randomuser.me/api/?results={num_customers}&nat=us'
    response = requests.get(url)
    print(response)
    data = response.json()['results']
    customers = []
    for person in data:
        first_name = person['name']['first']
        last_name = person['name']['last']
        dob = datetime.strptime(person['dob']['date'], '%Y-%m-%dT%H:%M:%S.%fZ').strftime('%m/%d/%Y')
        original_email = person['email']
        email = custom_domain(original_email, mode='mixed')
        gender = person['gender'].capitalize()
        state = person['location']['state']
        customer_type = random.choice(['Regular', 'Premium'])

        customers.append([
            first_name, last_name, dob, email, gender, state, customer_type
        ])
    return customers

# Generate 98,765 customer rows (5000 x 19 + 3765)
all_customers = []
for _ in range(19):
    all_customers.extend(get_customers(5000))
all_customers.extend(get_customers(3765))

In [ ]:
# Youngest Customer
pd.to_datetime(df['Birthday (mm/dd/yyyy)'], format='%m/%d/%Y').max()

In [ ]:
# Oldest Customer
pd.to_datetime(df['Birthday (mm/dd/yyyy)'], format='%m/%d/%Y').min()

In [ ]:
# Define the start and end dates
start_date = datetime.strptime('01/01/2012', '%m/%d/%Y')
end_date = datetime.strptime('12/31/2022', '%m/%d/%Y')

# Function to generate a random date in 2012
def random_date(start, end):
    delta = end - start
    random_days = random.randint(0, delta.days)
    return (start + timedelta(days=random_days)).strftime('%m/%d/%Y')

# Append a random registration date to each customer
for customer in all_customers:
    reg_date = random_date(start_date, end_date)
    customer.append(reg_date)

In [ ]:
for i, customer in enumerate(all_customers, start=1):
    customer_id = f"{i:010d}"  # Formats number as 10-digit string with leading zeros
    customer.insert(0, customer_id)

In [27]:
# all_customers = []

# with open('/content/drive/MyDrive/Colab Notebooks/customers.csv', mode='r', encoding='utf-8') as file:
#     reader = csv.reader(file)
#     next(reader)
#     for row in reader:
#         all_customers.append(row)

In [28]:
# Load State to State_ID mapping from states.csv
state_id_map = {}
with open('states.csv', mode='r', encoding='utf-8-sig') as file:
    reader = csv.DictReader(file)
    for row in reader:
        state_id_map[row['State_Desc']] = row['State_ID']

for customer in all_customers:
    state_name = customer[6]
    state_id = state_id_map.get(state_name, '')  # Default to empty if not found
    customer.insert(7, state_id)  # Insert after the State

In [30]:
city_map = {}

with open('cities.csv', mode='r', encoding='utf-8-sig') as file:
    reader = csv.DictReader(file)
    for row in reader:
        state_id = row['State_ID']
        city_id = row['City_ID']
        if state_id not in city_map:
            city_map[state_id] = []
        city_map[state_id].append(city_id)

updated_customers = []

for customer in all_customers:
    state_id = customer[7]
    del customer[6]

    possible_cities = city_map.get(state_id, [])
    if possible_cities:
        city_id = random.choice(possible_cities)
    else:
        city_id = None

    customer.insert(6, city_id)

    updated_customers.append(customer)

all_customers = updated_customers

In [32]:
columns = [
    'Customer_ID', 'First_Name', 'Last_Name', 'DOB', 'Email', 'Gender',
    'City_ID', 'State_ID', 'Customer_Type', 'Registration_Date'
]
# Convert your list of lists into a DataFrame
df = pd.DataFrame(all_customers, columns=columns)

# Remove duplicates based on the Email column
df.drop_duplicates(subset='Email', keep='first', inplace=True)

# Save to CSV
df.to_csv('customers.csv', index=False)

In [7]:
# Helper function: Generate random 20-character Transaction_ID
def generate_transaction_id(length=20):
    characters = string.ascii_uppercase + string.digits
    return ''.join(random.choices(characters, k=length))

# Helper function: Generate random datetime between two dates
def random_datetime(start, end):
    delta = end - start
    random_seconds = random.randint(0, int(delta.total_seconds()))
    return start + timedelta(seconds=random_seconds)

# Constants
transaction_types = ['Cash', 'Credit Card', 'Debit Card', 'PayPal']
num_transactions = 543210

start_date = datetime(2023, 1, 1, 0, 0, 0)
end_date = datetime(2023, 12, 31, 23, 59, 59)

# Track generated Transaction_IDs
generated_ids = set()

transactions = []

for _ in range(num_transactions):
    while True:
        transaction_id = generate_transaction_id()
        if transaction_id not in generated_ids:
            generated_ids.add(transaction_id)
            break

    transaction_attempt_no = 1
    transaction_datetime = random_datetime(start_date, end_date)
    transaction_type = random.choice(transaction_types)
    if transaction_type == 'Cash':
        transaction_status = 'Success'
    else:
        transaction_status = random.choices(['Success', 'Failure'], weights=[75, 25], k=1)[0]

    transactions.append([
        transaction_id,
        transaction_attempt_no,
        transaction_datetime.strftime('%m/%d/%Y %H:%M:%S'),
        transaction_type,
        transaction_status,
        '',  # Shipping_Method
        '',  # Transaction_Amount
        '',  # Shipping_Cost
        '',  # Discount_Percentage
        ''   # Final_Amount
    ])

    # Handle Failure: Maybe Retry
    current_datetime = transaction_datetime
    while transaction_status == 'Failure' and random.choice([True, False]):
        transaction_attempt_no += 1

        retry_start = current_datetime
        retry_end = current_datetime + timedelta(hours=1)
        if retry_end > end_date:
            break
        transaction_datetime = random_datetime(retry_start, retry_end)

        transaction_type = random.choice(transaction_types)
        if transaction_type == 'Cash':
            transaction_status = 'Success'
        else:
            transaction_status = random.choices(['Success', 'Failure'], weights=[75, 25], k=1)[0]

        transactions.append([
            transaction_id,
            transaction_attempt_no,
            transaction_datetime.strftime('%m/%d/%Y %H:%M:%S'),
            transaction_type,
            transaction_status,
            '',  # Shipping_Method
            '',  # Transaction_Amount
            '',  # Shipping_Cost
            '',  # Discount_Percentage
            ''   # Final_Amount
        ])

# Write to transactions.csv
with open('transactions.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow([
        'Transaction_ID', 'Transaction_Attempt_No', 'Transaction_Datetime', 'Transaction_Type', 'Transaction_Status',
        'Shipping_Method', 'Transaction_Amount', 'Shipping_Cost', 'Discount_Percentage', 'Final_Amount'
    ])
    writer.writerows(transactions)

In [8]:
print(len(generated_ids))

543210


In [ ]:
# Read Customer IDs
with open('customers.csv', mode='r', encoding='utf-8') as f:
    customers = list(csv.DictReader(f))
    customer_ids = [cust['Customer_ID'] for cust in customers]

# Read Product IDs and Prices
with open('products.csv', mode='r', encoding='utf-8') as f:
    products = list(csv.DictReader(f))
    product_price_map = {prod['Product_ID']: float(prod['Product_Price']) for prod in products}
    product_ids = list(product_price_map.keys())

# Read Transaction IDs
with open('transactions.csv', mode='r', encoding='utf-8') as f:
    transactions = list(csv.DictReader(f))
    transaction_ids = {txn['Transaction_ID'] for txn in transactions}

# Create Sales Data
sales_data = []
sale_counter = 1

for txn_id in transaction_ids:
    # Randomly pick a customer for this transaction
    customer_id = random.choice(customer_ids)

    # Randomly decide how many products the customer bought in this transaction (at least 1)
    num_products = random.randint(1, 5)
    chosen_products = random.sample(product_ids, min(num_products, len(product_ids)))

    for product_id in chosen_products:
        quantity = random.randint(1, 25)
        price_per_unit = product_price_map[product_id]
        total_cost = round(quantity * price_per_unit, 2)

        sale_id = str(sale_counter).zfill(10)
        sales_data.append({
            'Sale_ID': sale_id,
            'Customer_ID': customer_id,
            'Product_ID': product_id,
            'Product_Price': price_per_unit,
            'Quantity': quantity,
            'Cost': total_cost,
            'Transaction_ID': txn_id
        })

        sale_counter += 1

# Write to sales.csv
with open('sales.csv', mode='w', newline='', encoding='utf-8') as f:
    fieldnames = ['Sale_ID', 'Customer_ID', 'Product_ID', 'Product_Price',
                  'Quantity', 'Cost', 'Transaction_ID',]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(sales_data)

In [15]:
# Load sales data (to calculate total cost per transaction)
sales_df = pd.read_csv('sales.csv')

# Calculate total cost per transaction (sum of costs for each Transaction_ID)
transaction_costs = sales_df.groupby('Transaction_ID')['Cost'].sum().to_dict()

# Load transactions data
transactions_df = pd.read_csv('transactions.csv')

# Load customer data to get registration date and customer type
customers_df = pd.read_csv('customers.csv')

# Create a dictionary to map Customer_ID to their registration date and customer type
customer_info = {row['Customer_ID']: {'Registration_Date': row['Registration_Date'], 'Customer_Type': row['Customer_Type']}
                 for _, row in customers_df.iterrows()}

# Function to calculate Discount_Percentage based on conditions
def calculate_discount_percentage(customer_type, registration_date):
    registration_date = datetime.strptime(registration_date, '%m/%d/%Y')
    today = datetime.today()
    delta = today - registration_date
    years = delta.days // 365  # Approximate number of years

    if customer_type == 'Regular':
        if years < 1:
            return 0.0
        elif 1 <= years < 2:
            return 0.02
        elif 2 <= years < 5:
            return 0.04
        else:
            return 0.08
    elif customer_type == 'Premium':
        if years < 1:
            return 0.02
        elif 1 <= years < 2:
            return 0.05
        elif 2 <= years < 5:
            return 0.08
        else:
            return 0.12

# Create a dictionary to map Transaction_ID to customer_id (from sales.csv)
transaction_customer_map = sales_df.groupby('Transaction_ID')['Customer_ID'].first().to_dict()

# Add Transaction_Amount, Discount_Percentage, Shipping_Method and Shipping_Cost to the transactions dataframe
transaction_amounts = []
discount_percentages = []
shipping_methods = []
shipping_costs = []

for _, row in transactions_df.iterrows():
    transaction_id = row['Transaction_ID']

    # Add Transaction_Amount based on the sum of costs from sales data
    transaction_amount = round(transaction_costs.get(transaction_id, 0), 2)
    transaction_amounts.append(transaction_amount)

    # Get customer_id associated with this transaction from sales data
    customer_id = transaction_customer_map.get(transaction_id, None)

    # If the customer_id exists, get their registration date and customer type
    if customer_id:
        customer_info_row = customer_info.get(customer_id, {})
        registration_date = customer_info_row.get('Registration_Date', '01/01/1900')
        customer_type = customer_info_row.get('Customer_Type', 'Regular')

        # Calculate Discount_Percentage based on registration date and customer type
        discount_percentage = calculate_discount_percentage(customer_type, registration_date)
        discount_percentages.append(discount_percentage)
    else:
        # Default values if customer_id is not found (handle the case)
        transaction_amounts.append(0)
        discount_percentages.append(0)
    # --- Determine Shipping Method ---
    if customer_type == 'Premium':
        shipping_method = 'Express'
    else:  # Regular
        shipping_method = random.choice(['Standard', 'Express'])

    # --- Calculate Shipping Cost ---
    if shipping_method == 'Standard':
        if transaction_amount > 1500:
            shipping_cost = round(0.1 * transaction_amount, 2)
        elif transaction_amount >= 1000:
            shipping_cost = round(0.2 * transaction_amount, 2)
        else:
            shipping_cost = round(0.4 * transaction_amount, 2)

    elif shipping_method == 'Express':
        if customer_type == 'Regular':
            if transaction_amount > 1500:
                shipping_cost = round(0.2 * transaction_amount, 2)
            elif transaction_amount >= 1000:
                shipping_cost = round(0.4 * transaction_amount, 2)
            else:
                shipping_cost = round(0.65 * transaction_amount, 2)
        elif customer_type == 'Premium':
            if transaction_amount > 1500:
                shipping_cost = round(0.1 * transaction_amount, 2)
            elif transaction_amount >= 1000:
                shipping_cost = round(0.2 * transaction_amount, 2)
            else:
                shipping_cost = round(0.4 * transaction_amount, 2)

    shipping_methods.append(shipping_method)
    shipping_costs.append(shipping_cost)



# Add the new columns to the dataframe
transactions_df['Transaction_Amount'] = transaction_amounts
transactions_df['Discount_Percentage'] = discount_percentages
transactions_df['Shipping_Method'] = shipping_methods
transactions_df['Shipping_Cost'] = shipping_costs

# Calculate Final_Amount for all transactions
transactions_df['Final_Amount'] = transactions_df['Transaction_Amount'] * (1 - transactions_df['Discount_Percentage']) + transactions_df['Shipping_Cost']

# Function to round to nearest 0.05
def round_to_nearest_0_05(amount):
    return round(round(amount * 20) / 20, 2)

# Apply rounding rule
transactions_df['Final_Amount'] = transactions_df.apply(
    lambda row: round_to_nearest_0_05(row['Final_Amount']) if row['Transaction_Type'] == 'Cash' else round(row['Final_Amount'], 2),
    axis=1
)

# Save the updated transactions dataframe back to the transactions.csv file
transactions_df.to_csv('transactions.csv', index=False)